In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sujaykapadnis/emotion-recognition-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'emotion-recognition-dataset' dataset.
Path to dataset files: /kaggle/input/emotion-recognition-dataset


In [2]:
%ls /kaggle/input/emotion-recognition-dataset

data.csv  dataset/


In [3]:
%ls /kaggle/input/emotion-recognition-dataset/dataset/

Ahegao/  Angry/  Happy/  Neutral/  Sad/  Surprise/


In [4]:
import os
import shutil

In [5]:
import random

In [7]:
original_dir = "/kaggle/input/emotion-recognition-dataset/dataset/"
train_dir = 'dataset_split/train'
val_dir = 'dataset_split/val'

In [8]:
split_ratio = .1

os.makedirs(train_dir,exist_ok=True)
os.makedirs(val_dir,exist_ok=True)

In [9]:
for c in os.listdir(original_dir):
  cpath = os.path.join(original_dir,c)
  print(cpath)

/kaggle/input/emotion-recognition-dataset/dataset/Surprise
/kaggle/input/emotion-recognition-dataset/dataset/Angry
/kaggle/input/emotion-recognition-dataset/dataset/Neutral
/kaggle/input/emotion-recognition-dataset/dataset/Sad
/kaggle/input/emotion-recognition-dataset/dataset/Happy
/kaggle/input/emotion-recognition-dataset/dataset/Ahegao


In [10]:
for class_name in os.listdir(original_dir):
    class_path = os.path.join(original_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    split_index = int(len(images) * split_ratio)
    val_images = images[:split_index]
    train_images = images[split_index:]

    # Create subfolders for each class
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)


In [11]:
import os
import shutil
import random


# Make train and val directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for class_name in os.listdir(original_dir):
    class_path = os.path.join(original_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    split_index = int(len(images) * split_ratio)
    val_images = images[:split_index]
    train_images = images[split_index:]

    # Create subfolders for each class
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

    for img in train_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(train_dir, class_name, img)
        shutil.copy(src, dst)

    for img in val_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(val_dir, class_name, img)
        shutil.copy(src, dst)

    print(f"{class_name}: {len(train_images)} training images, {len(val_images)} validation images")

Surprise: 1111 training images, 123 validation images
Angry: 1182 training images, 131 validation images
Neutral: 3625 training images, 402 validation images
Sad: 3541 training images, 393 validation images
Happy: 3366 training images, 374 validation images
Ahegao: 1085 training images, 120 validation images


In [12]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=90,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   width_shift_range=0.2,
                                   height_shift_range=.2)

In [14]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(271,271),
                                                    batch_size=33,
                                                    class_mode='categorical')

Found 13910 images belonging to 6 classes.


In [17]:
val_generator = val_datagen.flow_from_directory(val_dir,
                                                    target_size=(271,271),
                                                    batch_size=33,
                                                    class_mode='categorical')

Found 1543 images belonging to 6 classes.


In [18]:
from tensorflow.keras.layers import Conv2D,Dense,Flatten,MaxPooling2D,Dropout
from tensorflow.keras.models import Sequential

In [19]:
model = Sequential()
model.add(Conv2D(64, 3, input_shape=(271, 271, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(.3))

model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(.3))

model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(.3))

model.add(Flatten())
model.add(Dense(6, activation='softmax'))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
fit = model.fit(train_generator,
                epochs=10,
                validation_data=val_generator)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 330s 746ms/step - accuracy: 0.2494 - loss: 1.7387 - val_accuracy: 0.2923 - val_loss: 1.6200
Epoch 2/10
358/422 ━━━━━━━━━━━━━━━━━━━━ 44s 694ms/step - accuracy: 0.2957 - loss: 1.5819